# Data Analyst Job Market Analysis

## Motivation
As a 3D artist with an informatics background, I analyze 
the data analyst job market to evaluate a possible career switch 
from 3D artist to data analyst.

## Part 1: USA Market Deep Dive
### 1.1 Dataset Overview
### 1.2 Data Cleaning 
### 1.2.1 Salary Estimate in Min, Max and Middle Salary
### 1.3 Salary Analysis
### 1.4 Skills Requirements
### 1.5 Geographic Distribution
### 1.6 Industry Breakdown

## Part 2: Germany Market Insights
### 2.1 Data Collection Methodology
"Daten manuell erhoben via StepStone, Indeed (Januar 2026)"
### 2.2 Salary Comparison (USA vs Germany)
### 2.3 Job Availability

## Part 3: Career Switch Analysis
### 3.1 3D Artist vs Data Analyst (Germany)
### 3.2 Transferable Skills
### 3.3 Conclusions & Next Steps

## Part 1: USA Market Deep Dive
### 1.1 Dataset Overview

In [3]:
import pandas as pd
import numpy as np

# 1. USA-Daten (dein Kaggle Dataset)
df_usa = pd.read_csv('data/DataAnalyst.csv',delimiter=',')

df_usa.head(1)



,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True


### 1.2 Data Cleaning 
### 1.2.1 Salary Estimate in Min, Max and Middle Salary

In [ ]:
df_usa_cleaned = df_usa.copy()

df_usa_cleaned['Salary Estimate']

# for sep in ["K", "$", ","]:
#     df_usa_cleaned['Salary Estimate'] = df_usa_cleaned['Salary Estimate'].replace(sep, " ")
# Cleara the Sybmols 'K', '$' and '-'
df_usa_cleaned['Salary Estimate'] = df_usa_cleaned['Salary Estimate'].str.replace('K', ' ')
df_usa_cleaned['Salary Estimate'] = df_usa_cleaned['Salary Estimate'].str.replace('$', ' ')
df_usa_cleaned['Salary Estimate'] = df_usa_cleaned['Salary Estimate'].str.replace('-', '')

# Split the Min Salary Estimate and Max Salary Estimate 
df_usa_cleaned['Min Salary Estimate'] = df_usa_cleaned['Salary Estimate'].str.split(' ').str[1]
df_usa_cleaned['Max Salary Estimate'] = df_usa_cleaned['Salary Estimate'].str.split(' ').str[3]


# The Null Values from the Min Salary Estimate and Max Salary Estimate, 
# will be settet to the middle Values of the collum
Min_Salary_middle_Value = df_usa_cleaned['Min Salary Estimate'].mode()[0]
df_usa_cleaned.loc[df_usa_cleaned['Min Salary Estimate'].isnull(), 'Min Salary Estimate' ] = Min_Salary_middle_Value

Min_Salary_middle_Value = df_usa_cleaned['Max Salary Estimate'].mode()[0]
df_usa_cleaned.loc[df_usa_cleaned['Max Salary Estimate'].isnull(), 'Max Salary Estimate' ] = Min_Salary_middle_Value

# #df_usa_cleaned.loc[[2149]]
df_usa_cleaned.drop('Salary Estimate', axis=1, inplace=True)
# df_usa_cleaned.drop(df_usa_cleaned[df_usa_cleaned['Min Salary Estimate'].isnull()], axis=1, inplace=True)  

# Set the type of the collums to int
df_usa_cleaned['Min Salary Estimate'] = df_usa_cleaned['Min Salary Estimate'].astype(int)
df_usa_cleaned['Max Salary Estimate'] = df_usa_cleaned['Max Salary Estimate'].astype(int)

# df_usa_cleaned.info()
df_usa_cleaned.head(1)

,Unnamed: 0,Job Title,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,Min Salary Estimate,Max Salary Estimate
0,0,"Data Analyst, Center on Immigration and Justic...",Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,37,66


In [ ]:
import pandas as pd
import numpy as np
# 1. ########################################### !!!!!!!!!!!!
df = pd.read_excel('flight_price.xlsx')
# 2. ########## EDI ############################ !!!!!!!!!!!!
# df.info()
# df.columns
# df['Airline'].describe()
# df.isnull().sum()
#df.head(2)
# df['Route'].unique()
# df[df['Route'].isnull()]
df_copy = df.copy()
df_copy[df_copy['Route'].isnull()]
################################################ !!!!!!!!!!!!
# 3.1 ##### Missing Values ##################### !!!!!!!!!!!!
############ Ezetzen mit Mode ################## !!!!!!!!!!!!
Total_Stops_mode = df_copy['Total_Stops'].mode()[0]
df_copy.loc[df_copy['Total_Stops'].isnull(), 'Total_Stops' ] = Total_Stops_mode
################################################ !!!!!!!!!!!!
Route_mode = df_copy['Route'].mode()[0]
df_copy.loc[df['Route'].isnull(), 'Route'] = Route_mode
################################################ !!!!!!!!!!!!
df_copy.loc[9039]
# 3.2 ########################################## !!!!!!!!!!!!
############# Droping an entry ################# !!!!!!!!!!!!
df_copy.drop(index=9039, inplace=True)
df_copy.loc[9039]
################################################ !!!!!!!!!!!!
# 4.1 ### Count the most populer destination ### !!!!!!!!!!!!
df_copy['Destination'].unique()
Destination_Mode = df_copy['Destination'].mode()[0]
[df_copy['Destination'] == 'Cochin'] ## !!!! Print Out All Cochin destinations #
print((df_copy['Destination'] == Destination_Mode).sum())
print the sum of all most comming Destinations
################################################ !!!!!!!!!!!!
### Duration ################################### !!!!!!!!!!!!
Duration_mode = df_copy['Duration'].mode()[0]
df['Duration'].unique()
Duration_mode
print( (df_copy['Duration'] == Duration_mode).sum() )
################################################ !!!!!!!!!!!!
df_copy = df.copy()
### Data Cleaning ############################## !!!!!!!!!!!!
### Dep_Time ############################## !!!!!!!!!!!!
df_copy['Dep_Houer'] = df_copy['Dep_Time'].str.split(':').str[0]
df_copy['Dep_Min'] = df_copy['Dep_Time'].str.split(':').str[1]
################################################ !!!!!!!!!!!!
df_copy.drop('Dep_Time', axis=1 , inplace=True)################################################ !!!!!!!!!!!!
df_copy['Dep_Houer'] = df_copy['Dep_Houer'].astype(int)
df_copy['Dep_Min'] = df_copy['Dep_Min'].astype(int)
################################################ !!!!!!!!!!!!
### Date_of_Journey ############################ !!!!!!!!!!!!
df_copy['DoJ_Day'] = df_copy['Date_of_Journey'].str.split('/').str[0]
df_copy['DoJ_Months'] = df_copy['Date_of_Journey'].str.split('/').str[1]
df_copy['DoJ_Year'] = df_copy['Date_of_Journey'].str.split('/').str[2]
################################################ !!!!!!!!!!!!
df_copy.drop('Date_of_Journey', axis=1, inplace=True)
################################################ !!!!!!!!!!!!
df_copy['DoJ_Months'] = df_copy['DoJ_Months'].astype(int)
df_copy['DoJ_Year'] = df_copy['DoJ_Year'].astype(int)
df_copy['DoJ_Day'] = df_copy['DoJ_Day'].astype(int)
############### Ploting ####################### !!!!!!!!!!!!
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(5,5))
# sns.set_theme(style="whitegrid")
# sns.boxplot(
# data=df_copy,
# x = 'Price',
# y = 'Total_Stops'
# )
# plt.show()
################################################ !!!!!!!!!!!!
# sns.boxenplot(
# data=df_copy,
# x = 'Price',
# y = 'Total_Stops'
# )
# plt.show()


In [27]:
# 2. Deutschland - Manuelle Daten (ergänzend)
germany_data = {
    'Country': ['Germany'] * 4,
    'City': ['Düsseldorf'] * 4,
    'Job_Title': ['Data Analyst', 'Data Analyst', '3D Artist', '3D Artist'],
    'Level': ['Entry', 'Senior', 'Entry', 'Senior'],
    'Avg_Salary_EUR': [50100, 80000, 36400, 50000],
    'Job_Count_Duesseldorf': [497, np.nan, 4, np.nan],
    'Source': ['StepStone 2026'] * 4
}

df_germany = pd.DataFrame(germany_data)

# 3. Währungsumrechnung für Vergleich
exchange_rate = 0.92  # USD to EUR (aktuell anpassen)

df_germany



,Country,City,Job_Title,Level,Avg_Salary_EUR,Job_Count_Duesseldorf,Source
0,Germany,Düsseldorf,Data Analyst,Entry,50100,497.0,StepStone 2026
1,Germany,Düsseldorf,Data Analyst,Senior,80000,NaN,StepStone 2026
2,Germany,Düsseldorf,3D Artist,Entry,36400,4.0,StepStone 2026
3,Germany,Düsseldorf,3D Artist,Senior,50000,NaN,StepStone 2026
